# Proyecto DATAMAD

## 1. Carga de datos

## 1.1 Carga de datos de Idealista

En primer lugar importamos las librerías:

In [9]:
import requests

## Esta librería no te servirá en este análisis, pero
## es la librería básica para scrapear páginas más sencillas

from bs4 import BeautifulSoup as bs

## Esta sí. Con ella, sacaremos datos del HTML.

import random

## Para usar números aleatorios. Esta no era tan complicada.
## La usaremos para no usar el mismo tiempo siempre al scrapear.

import time

## Con esta metemos el retardo a la máquina. Las máquinas van rápidas. Yo no.

import pandas as pd
import numpy as np

## EL bread and butter del Data Science. Tablas y operaciones aritméticas.

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

## Esta es la manera para usar selenium normal, pero nosotros tenemos que parecer humanos.
## Por eso, usamos la siguiente librería.

import undetected_chromedriver as uc

## La persona que desarrolló esto es un crack. Configuró todos los proxies y
## demás (digo demás porque no se qué brujería ha usado) para ser indetectable.
## Usaremos este browser.

Iniciamos el explorador, en este caso utilizamos Chrome

In [10]:
browser = uc.Chrome()

En esta sección extraemos de la página de idealista deseada los ids de las viviendas.

In [49]:
import asyncio
import aiohttp
import random
import time
import pandas as pd  # Para exportar los datos a CSV
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc

# Configuración de Selenium con undetected_chromedriver
def get_driver():
    options = uc.ChromeOptions()
    options.add_argument('--headless')  # No mostrar la ventana del navegador
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    # Crea el navegador sin ser detectado
    driver = uc.Chrome(options=options)
    return driver

# Función para extraer los IDs de una página de manera asíncrona
async def fetch_page(session, url):
    # Simulamos un navegador real con el User-Agent adecuado
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
    }

    async with session.get(url, headers=headers) as response:
        content = await response.text()
        return url, response.status, content

# Función para extraer los IDs de cada página HTML
def extract_ids_from_html(html_content):
    soup = bs(html_content, 'lxml')
    try:
        articles = soup.find('main', {'class': 'listing-items'}).find_all('article')
        ids = [article.get('data-element-id') for article in articles if article.get('data-element-id') is not None]
        return ids
    except AttributeError:
        return []

# Función para procesar múltiples URLs en paralelo
async def fetch_urls_in_parallel(url_list):
    ids = []
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_page(session, url) for url in url_list]
        for task in asyncio.as_completed(tasks):
            url, status_code, content = await task
            if status_code == 200:
                page_ids = extract_ids_from_html(content)
                ids.extend(page_ids)
                print(f"Extracted {len(page_ids)} IDs from {url}")
            else:
                print(f"Failed to fetch {url} with status code {status_code}")
    return ids

# Función para manejar el scraping completo
async def main_scraper():
    # Empezamos con la creación del navegador (Selenium con undetected_chromedriver)
    browser = get_driver()

    # Lista de URLs a scrapear
    x = 1
    url_list = []

    # Crear las URLs para las primeras 5 páginas
    while True:
        url = f'https://www.idealista.com/venta-viviendas/madrid-madrid/pagina-{x}.htm'
        url_list.append(url)
        x += 1
        if x > 5:  # Limitar a las primeras 5 páginas para la prueba
            break

    # Usamos Selenium para aceptar cookies si es necesario
    for url in url_list:
        browser.get(url)
        time.sleep(random.randint(10, 12))  # Retardo para parecer humano
        try:
            browser.find_element(By.XPATH, '//*[@id="didomi-notice-agree-button"]').click()
        except:
            pass

    # Ahora, obtenemos las páginas de manera asíncrona
    ids = await fetch_urls_in_parallel(url_list)

    # Mostrar los resultados finales
    ids = list(set(ids))  # Eliminar duplicados
    print(f"Total IDs collected: {len(ids)}")
    print(ids)

    # Cerrar el navegador al final
    browser.quit()

    # Convertir los IDs en un DataFrame
    df_ids = pd.DataFrame(ids, columns=["ID"])

    # Exportar los IDs a un archivo CSV
    export_to_csv(df_ids)

    # Devolver el DataFrame para usarlo en otras partes del código
    return df_ids

# Función para exportar los IDs a un archivo CSV
def export_to_csv(df, filename="ids.csv"):
    df.to_csv(filename, index=False)  # Exportar a un CSV sin el índice
    print(f"Exported {len(df)} IDs to {filename}")

# Ejecutar el scraper en el bucle de eventos actual
if __name__ == "__main__":
    # Para Jupyter o IPython, no usamos asyncio.run(), simplemente usamos await
    df_ids = await main_scraper()

    # Ahora puedes usar df_ids en otras partes del código
    print("DataFrame with extracted IDs:")
    print(df_ids)



could not detect version_main.therefore, we are assuming it is chrome 108 or higher


Extracted 30 IDs from https://www.idealista.com/venta-viviendas/madrid-madrid/pagina-5.htm
Extracted 30 IDs from https://www.idealista.com/venta-viviendas/madrid-madrid/pagina-3.htm
Extracted 30 IDs from https://www.idealista.com/venta-viviendas/madrid-madrid/pagina-2.htm
Extracted 30 IDs from https://www.idealista.com/venta-viviendas/madrid-madrid/pagina-4.htm
Extracted 30 IDs from https://www.idealista.com/venta-viviendas/madrid-madrid/pagina-1.htm
Total IDs collected: 149
['104440887', '103666203', '104608513', '105392078', '106132709', '106420775', '106054242', '105453336', '104161105', '104952998', '104663736', '102911946', '105036254', '106264579', '105070649', '105843197', '105774021', '105976873', '105890453', '106091596', '104613472', '103839118', '105386768', '106137924', '103784576', '105075962', '98249348', '105107064', '106135117', '105345023', '106330563', '102711209', '98352583', '103610280', '106300829', '106093199', '106155964', '105155727', '94336938', '105962129', '1

Hacemos un data frame con los datos.

In [50]:
# Usando el DataFrame después de obtenerlo
df_casas = df_ids  # Puedes usar directamente el DataFrame de IDs aquí

# Hacer algo con el DataFrame
print(df_casas.head())


          ID
0  104440887
1  103666203
2  104608513
3  105392078
4  106132709


Y lo exportamos a un csv

In [51]:
casas = pd.Series()

A continuación, a partir del id de las casas, extraemos el resto de datos utilizando BeautifulSoup

In [67]:
def parsear_inmueble(id_inmueble):
    
    print( '\n Casa numero: ' + id_inmueble)
    
    url = "https://www.idealista.com/inmueble/" + id_inmueble + "/"
    
    browser.get(url)

    html = browser.page_source
    
    soup = bs(html, 'lxml')

    titulo = soup.find('span',{'class':'main-info__title-main'}).text
    
    print('\n Titulo: ' + titulo)
    
    localizacion = soup.find('span',{'class':'main-info__title-minor'}).text.split(',')[0]

    print('\n Localizacion: ' + localizacion)
    precio = int(soup.find('span',{'class':'txt-bold'}).text.replace('.',''))

    c1 = soup.find('div',{'class':'details-property'}).find('div',{'class': 'details-property-feature-one'})

    caract_basicas = [caract.text.strip() for caract in c1.find_all('li')]
    
    #print('Caracteristicas basicas:' + caract_basicas)

    c2 = soup.find('div',{'class':'details-property'}).find('div',{'class': 'details-property-feature-two'})

    caract_extra = [caract.text.strip() for caract in c2.find_all('li')]
    
    #print('Caracteristicas extras:' + caract_extra)
    
    casas['titulo'] = titulo
    
    casas['localizacion'] = localizacion
    
    casas['precio'] = precio
    
    casas['caracteristicas_basicas'] = caract_basicas
    
    casas['caracteristicas_extras'] = caract_extra
    
    df_casas = pd.DataFrame(casas)
    
    return(df_casas.T)

In [68]:


# Inicializa el DataFrame vacio donde almacenaremos los datos de todos los inmuebles
df_casas_completas = pd.DataFrame()

# Itera sobre cada ID en el DataFrame df_casas
for i in range(len(df_casas)):
    # Obtén el ID de la fila actual
    id_inmueble = df_casas.iloc[i]['ID']
    
    # Llama a la función parsear_inmueble con el ID
    df_inmueble = parsear_inmueble(id_inmueble)
    
    # Concatena el DataFrame actual con el DataFrame de los nuevos datos
    df_casas_completas = pd.concat([df_casas_completas, df_inmueble], ignore_index=True)
    
    # Espera un tiempo aleatorio entre 4 y 8 segundos para simular comportamiento humano
    time.sleep(random.randint(4, 8))

# Al finalizar, df_casas_completas tendrá los datos de todos los inmuebles
print(df_casas_completas)



 Casa numero: 104440887

 Titulo: Piso en venta en paseo del General Martínez Campos

 Localizacion: Almagro

 Casa numero: 103666203

 Titulo: Piso en venta en avenida de América, 37

 Localizacion: Prosperidad


KeyboardInterrupt: 

In [73]:
df_casas_completas.reset_index(drop=True, inplace=True)

Y exportamos todos los datos a otro csv.

In [74]:
df_casas_completas.to_csv('casas_idealista.csv', index = False, sep = ';', encoding = 'utf-16')

## 1.2 Carga de datos DATAMAD

Datos de centros sanitarios

In [9]:
# Importamos las librerías necesarias
import csv
import requests
from pyproj import Proj, transform

# URL del archivo CSV
base_url = "https://datos.comunidad.madrid/catalogo/dataset/d8a0a444-adf5-4c04-8999-0eac3de52cb7/resource/2948b4da-8b39-42b7-b667-779a5284f39d/download/centros_servicios_establecimientos_sanitarios.csv"

# Realizamos la petición
result = requests.get(base_url)

# Si la conexión es exitosa, procesamos el CSV
if result.status_code == 200:
    # Convertimos el texto del CSV en una lista de diccionarios, con ';' como delimitador
    csv_data = result.text.splitlines()
    reader = csv.DictReader(csv_data, delimiter=';')

    # Filtramos solo los datos del municipio de Madrid y eliminamos duplicados por 'centro_nro_registro'
    seen_centers = set()
    filtered_data = []
    for row in reader:
        centro_nro_registro = row.get("centro_nro_registro")
        if row.get("municipio_nombre") and row["municipio_nombre"].lower() == "madrid" and centro_nro_registro not in seen_centers:
            seen_centers.add(centro_nro_registro)  # Agregamos el centro para evitar duplicados
            filtered_data.append(row)

    # Configuramos el sistema de coordenadas UTM y WGS84 (latitud/longitud)
    utm_proj = Proj(proj='utm', zone=30, ellps='WGS84')  # UTM Zone 30T (España)
    wgs84_proj = Proj(proj='latlong', datum='WGS84')

    # Convertimos las coordenadas UTM a latitud y longitud
    for row in filtered_data:
        try:
            # Verificamos si las coordenadas UTM están presentes y no están vacías
            if row["localizacion_coordenada_x"] and row["localizacion_coordenada_y"]:
                # Convertimos las coordenadas UTM a float
                utm_x = float(row["localizacion_coordenada_x"])
                utm_y = float(row["localizacion_coordenada_y"])
                
                # Convertimos a latitud y longitud
                lon, lat = transform(utm_proj, wgs84_proj, utm_x, utm_y)
                
                # Añadimos las nuevas coordenadas al diccionario
                row["latitud"] = lat
                row["longitud"] = lon
            else:
                # Si las coordenadas están vacías, asignamos None o dejamos en blanco
                row["latitud"] = None
                row["longitud"] = None

            # Eliminamos la columna 'oferta_asistencial' si existe
            row.pop("oferta_asistencial", None)

        except Exception as e:
            print(f"Error al convertir coordenadas para la fila: {row['centro_nro_registro']}", e)

    # Guardamos los datos filtrados y transformados en un nuevo archivo CSV
    with open("datos_madrid_latlon.csv", mode="w", newline="", encoding="utf-8") as file:
        # Revisamos si 'latitud' y 'longitud' ya están en las claves y los agregamos solo si no están
        fieldnames = list(filtered_data[0].keys())
        if "latitud" not in fieldnames:
            fieldnames.append("latitud")
        if "longitud" not in fieldnames:
            fieldnames.append("longitud")

        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(filtered_data)
    
    print("Archivo 'datos_madrid_latlon.csv' generado exitosamente.")
else:
    print("Error en la solicitud:", result.status_code)


C:\Users\carme\AppData\Local\Temp\ipykernel_11968\87057601.py:41: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(utm_proj, wgs84_proj, utm_x, utm_y)


Archivo 'datos_madrid_latlon.csv' generado exitosamente.


# 2. Limpieza de datos y coordenadas

Ahora procedemos al tratamiento de los datos. 

In [28]:
import pandas as pd
import re

# Cargar el archivo CSV
df = pd.read_csv("casas_idealista.csv", sep=";", encoding="utf-16")

# Convertir 'caracteristicas_basicas' en una lista de Python, si no está ya en ese formato
df['caracteristicas_basicas'] = df['caracteristicas_basicas'].apply(eval)

# Función para extraer el número de calle
def extraer_numero_calle(titulo):
    # Busca un número en la dirección que esté seguido de coma o espacio
    match = re.search(r',\s?(\d+)', titulo)
    return int(match.group(1)) if match else None

# Aplicar la función para crear la columna 'numero_calle'
df['numero_calle'] = df['titulo'].apply(extraer_numero_calle)

# Función para extraer los datos de 'caracteristicas_basicas'
def extraer_datos(lista):
    datos = {
        'metros_cuadrados': None,
        'habitaciones': None,
        'baños': None,
        'parcela_m2': None,
        'plaza_garaje': None,
        'estado': None,
        'orientacion': None,
        'año_construccion': None,
        'calefaccion': None,
        'planta': None,
        'ascensor': None,
    }

    for item in lista:
        # Solo procesa elementos que contengan características específicas de la vivienda
        if re.search(r'\bm²\b|\bhabitaciones\b|\bbaños\b|garaje|estado|Orientación|Construido en|Calefacción|Planta|ascensor|Parcela', item):
            if re.search(r'\d+\s?m² construidos', item):
                datos['metros_cuadrados'] = int(re.search(r'\d+', item).group())
            elif 'habitaciones' in item:
                datos['habitaciones'] = int(re.search(r'\d+', item).group())
            elif 'baños' in item:
                datos['baños'] = int(re.search(r'\d+', item).group())
            elif 'Parcela de' in item:
                datos['parcela_m2'] = int(re.search(r'\d+', item.replace('.', '')).group())
            elif 'garaje' in item:
                datos['plaza_garaje'] = True
            elif 'Segunda mano' in item or 'para reformar' in item:
                datos['estado'] = item
            elif 'Orientación' in item:
                datos['orientacion'] = item.split(' ')[-1]
            elif 'Construido en' in item:
                datos['año_construccion'] = int(re.search(r'\d+', item).group())
            elif 'Calefacción' in item:
                datos['calefaccion'] = item.split(': ')[-1]
            elif 'Planta' in item:
                datos['planta'] = item
            elif 'ascensor' in item:
                datos['ascensor'] = True

    return datos

# Aplicar la función y expandir los resultados en el DataFrame
datos_df = df['caracteristicas_basicas'].apply(extraer_datos).apply(pd.Series)

# Combinar los datos originales con las nuevas columnas
df = pd.concat([df, datos_df], axis=1)

# Guardar el DataFrame en un nuevo archivo CSV si se desea
df.to_csv('casas_completo.csv', index=False, sep=';')
# Guardar el DataFrame en un nuevo archivo CSV con punto y coma como delimitador



# Mostrar el DataFrame final
print(df.head())


                                              titulo localizacion   precio  \
0          Piso en venta en calle del Duque de Sesto         Goya  1290000   
1          Piso en venta en calle del Duque de Sesto         Goya  1290000   
2                          Ático en venta en Adelfas       Retiro   880000   
3   Chalet pareado en venta en calle Virgen de lo...      Aravaca  2280000   
4   Casa o chalet independiente en venta en calle...   Valdemarín  3500000   

                             caracteristicas_basicas  \
0  [133 m² construidos, 2 habitaciones, 3 baños, ...   
1  [133 m² construidos, 2 habitaciones, 3 baños, ...   
2  [175 m² construidos, 4 habitaciones, 3 baños, ...   
3  [Chalet pareado, 505 m² construidos, 7 habitac...   
4  [Casa o chalet independiente, 749 m² construid...   

                              caracteristicas_extras  numero_calle  \
0   ['Aire acondicionado', 'Consumo:', 'Emisiones:']           NaN   
1   ['Aire acondicionado', 'Consumo:', 'Emisiones:']  

Añadimos las coordenadas a las direcciones utilizando DireccionesVigentes.csv

In [26]:
import pandas as pd
import re

# Cargar los datos
direcciones_df = pd.read_csv("DireccionesVigentes.csv", encoding="latin1", delimiter=";")
casas_df = pd.read_csv("casas_completo.csv", encoding="utf-8", delimiter=";")

# Limpiar y normalizar nombres de calles en DireccionesVigentes
direcciones_df["VIA_NOMBRE"] = direcciones_df["VIA_NOMBRE"].str.upper().str.strip()

# Eliminar duplicados en direcciones_df para que solo haya un valor de coordenadas por calle
direcciones_df = direcciones_df.drop_duplicates(subset=["VIA_NOMBRE"], keep="first")

# Función para extraer y normalizar el nombre de la calle del título
def extraer_nombre_calle(titulo):
    # Buscar el tipo de vía seguido de la última palabra del nombre de la calle
    match = re.search(r'\b(CALLE|AVENIDA|PASEO|PLAZA|RONDA|CAMINO)\s+(?:DE(L|LA)?\s+)?(\w+)$', titulo, re.IGNORECASE)
    if match:
        return match.group(3).upper().strip()  # Solo capturar la última palabra
    return None

# Crear una columna temporal en casas_df para almacenar el nombre de la calle extraído
casas_df["CALLE_EXTRAIDA"] = casas_df["titulo"].apply(extraer_nombre_calle)

# Hacer el merge de ambas bases de datos en función del nombre de la calle
casas_completo = casas_df.merge(direcciones_df[['VIA_NOMBRE', 'LATITUD', 'LONGITUD']],
                                left_on="CALLE_EXTRAIDA", right_on="VIA_NOMBRE", how="left")

# Eliminar la columna temporal y renombrar columnas
casas_completo.drop(columns=["CALLE_EXTRAIDA", "VIA_NOMBRE"], inplace=True)

# Guardar el archivo actualizado
casas_completo.to_csv("casas_completo_actualizado.csv", index=False)
print("Archivo 'casas_completo_actualizado.csv' creado con éxito.")



Archivo 'casas_completo_actualizado.csv' creado con éxito.


Aquí eliminamos todos aquellos pisos de los cuales no ha sido posible obtener las coordenadas.

In [32]:
casas_con_coordenadas = casas_completo.dropna(subset=['LATITUD', 'LONGITUD'])

# Guardar el archivo de casas con coordenadas
casas_con_coordenadas.to_csv("casas_con_coordenadas.csv", index=False)
print("Archivo 'casas_con_coordenadas.csv' creado con éxito.")

Archivo 'casas_con_coordenadas.csv' creado con éxito.


In [35]:
import pandas as pd
import re

# Definir una función para convertir coordenadas de DMS a decimal
def dms_a_decimal(dms):
    # Separar grados, minutos, segundos y dirección
    degrees, minutes, seconds, direction = re.split("[°'\" ]+", dms.strip())
    decimal = float(degrees) + float(minutes) / 60 + float(seconds) / 3600
    if direction in ['S', 'W']:
        decimal = -decimal
    return decimal

# Cargar los datos desde un CSV
casas_df = pd.read_csv("casas_con_coordenadas.csv", encoding="utf-8", delimiter=",")

# Imprimir los nombres de las columnas para depuración
print("Columnas disponibles en el DataFrame:", casas_df.columns.tolist())

# Convertir las coordenadas de LATITUD y LONGITUD a decimal
# Asegúrate de que los nombres coincidan exactamente con los nombres de columna
if 'LATITUD' in casas_df.columns and 'LONGITUD' in casas_df.columns:
    casas_df['LATITUD'] = casas_df['LATITUD'].apply(dms_a_decimal)
    casas_df['LONGITUD'] = casas_df['LONGITUD'].apply(dms_a_decimal)
else:
    print("No se encontraron las columnas 'LATITUD' y/o 'LONGITUD' en el DataFrame.")

# Guardar el nuevo archivo CSV con las coordenadas transformadas
casas_df.to_csv("casas_con_coordenadas_transformadas.csv", index=False, sep=';')
print("Archivo 'casas_con_coordenadas_transformadas.csv' creado con éxito.")



Columnas disponibles en el DataFrame: ['titulo', 'localizacion', 'precio', 'numero_calle', 'metros_cuadrados', 'habitaciones', 'baños', 'parcela_m2', 'plaza_garaje', 'estado', 'orientacion', 'año_construccion', 'calefaccion', 'planta', 'ascensor', 'LATITUD', 'LONGITUD']
Archivo 'casas_con_coordenadas_transformadas.csv' creado con éxito.


# 3. Aplicacion